# Pybullet tutorial - Part. 1
by SeoYoung Son
##### Reference
* <a href = "https://m.blog.naver.com/PostView.nhnblogId=einsbon&logNo=221294095704&proxyReferer=https%3A%2F%2Fwww.google.com%2F">네이버 블로그: 미래 가젯 연구소</a>
* <a href = "https://docs.google.com/document/d/10sXEhzFRSnvFcl3XxNGhnD4N2SedqwdAvK3dsihxVUA/edit#heading=h.2ye70wns7io3">Pybullet Quickstart Guide</a>
* <a href = "https://medium.com/@gerardmaggiolino/creating-openai-gym-environments-with-pybullet-part-1-13895a622b24">Creating OpenAI Gym Environments with Pybullet</a>
* <a href = "http://wiki.ros.org/urdf/Tutorials">URDF tutorials</a>

#### 이번 시간 학습 목표:
* Pybullet에 대해 간단히 알아보는 시간을 갖는다
* ROS에 들어가는 모델인 URDF를 작성해본다.   

본 문서는 SAI 스터디용 자료이므로 허락없이 퍼가시는 것을 금합니다

## Getting started..
이 문서는 어느 정도의 파이썬 지식과 기초적인 딥러닝 지식을 보유한 사람들을 위해 작성하였습니다.  
아래 준비물들을 준비하심 좋을 것 같습니다.  
* Pycharm (2018.02 이상)
* Python 3.7 ver.
* Notepad++ 혹은 VisualCode

cmd 및 터미널을 열어서 아래 모듈들을 설치해주세요!
```cmd
pip install pybullet
pip install tensorflow         //필요하면 설치
pip install baselines          //현재 tensorflow2.0이 반영이 안되어 RL의 deepQ 기능 사용 불가
```

이제 준비를 모두 다 끝마치셨다면 아래 코드를 복사해서 잘 돌아가는지 확인해봅시다.  
각 코드가 무엇을 의미하는지 차근차근 설명해드리겠습니다.

In [2]:
import pybullet as p
import time
import pybullet_data

ModuleNotFoundError: No module named 'pybullet'

In [ ]:
physicsClient = p.connect(p.GUI)
p.setAdditionalSearchPath(pybullet_data.getDataPath())
p.setGravity(0, 0, -9.8)
p.setTimeStep(1/200)

planeID = p.loadURDF("plane.urdf")
cubeStartPos = [2, -1, 1]
cubeStartOrientation = p.getQuaternionFromEuler([0, 0, 3.14])
robotID = p.loadURDF("r2d2.urdf", cubeStartPos, cubeStartOrientation)

for i in range(10000):
    p.stepSimulation()
    time.sleep(1 / 200)
p.disconnect(physicsClient)

* ```p.connect(p.GUI)```: 화면에 출력하게 연결하는 함수입니다. p.GUI 대신 p.DIRECT를 넣으면 시뮬레이션이 화면으로 출력되지 않습니다.
* ```p.setAdditionalSearchPath()```: 모델의 경로를 불러옵니다.
* ```p.setGravity(0,0,-9,8)```: 중력 설정, 보통 (0,0,-9.8) 내지는 (0,0,-10)으로 설정합니다
* ```p.setTimeStep(1/200)```: time = time + delta_time이라고 했을 대 delta_time이라고 생각하심될 것 같습니다.
* ```p.loadURDF("URDF_PATH", position, orientation)```: urdf을 불러오고 initial position과 orientation을 지정합니다.

위 코드를 실행하여 로봇이 나왔다면 설치에 성공하신 겁니다!

## URDF 작성법

이번에는 URDF 작성하는 방법에 대해 알아보겠습니다.  
URDF는 Unified Robotic Description Format 의 약어로 ROS에 사용되는 xml 파일 포맷입니다.  
이외 Pybullet에서는 SDF와 같은 기타 포맷도 사용되지만 이번 시간에는 URDF에 대해서만 간략하게 알아보겠습니다. 

간단한 자동차를 손으로 직접 짜보는 시간을 가질 것입니다.  
Notepad++을 켜서 아래 코드를 보시면서 차근차근 따라하시면 될 것 같습니다.

### 가장 기본 Frame 만들기 

URDF의 구조는 아래 이미지와 같습니다.
<img src="./urdf_structure.png" width="300" height="300">
</br></br>

기본적으로 **link**와 **link** 태그 사이 **joint**를 넣는 방식입니다.  
**link** 하위 태그로는 크게 
1. 모니터에서 보이는 부분을 담당하는 **visual**,  
2. Boundary box의 역할을 하는 **collision**
3. 물리적 속성을 담당하는 **inertial**
크게 세 가지로 나뉩니다. 각각의 태그들에서도 또다른 태그들이 존재하지만,  
일단은 이렇게 알아두고 아래 코드를 보면서 넘어갑시다..!

```xml
<?xml version="1.0"?> 
<robot name="simplecar">         //로봇 아이디이자 이름을 정합니다. 웬만해선 urdf파일명과 동일하게 해주세요!
    <!-- Base Frame of Car --> 
    <link name="base_link">      //로봇의 부분인 link 입니다. ID가 base_link 입니다.
        <visual>                 //시각적인 부분을 담당하는 부분입니다. 
            <geometry>           //상자를 만들기 위해 geometry 태그를 적어줍시다.
                <box size="0.5 0.3 0.1"/>  //x = 0.5 y = 0.3 z = 0.1 크기만큼 지정해줍니다
            </geometry>          //항상 태그는 "/" 형태로 끝맺음합니다
        </visual> 
    </link>
</robot>                 
```

※ 편의상 주석을 //라고 했습니다. xml의 주석은 ```<!-- -->```와 같습니다!

geometry 하위 태그로 여러가지가 존재합니다.
<a href = "http://wiki.ros.org/urdf/XML/link">사이트</a>에 방문하셔서 더 자세한 것들을 확인할 수 있습니다.  
```<mesh filename = "car.obj" scale="0.1 0.1 0.1"/>``` 와 같이 3D 오브젝트 파일을 urdf로 가공해서 만들어낼 수도 있습니다.

#### 💻연습 코딩
차의 몸통을 만들었으니, 차 바퀴 네 부분을 다음 요구 사항에 맞추어 만들어봅시다..!

##### Left Back Wheel
* link name: left_back_wheel
* geometry: 원기둥이고 길이는 0.05, 반지름은 0.1입니다.

##### Right Back Wheel
* link name: right_back_wheel
* geometry: 원기둥이고 길이는 0.05, 반지름은 0.1입니다.

### Joint: link와 link 사이 연결하기

위에서 만들었던 바퀴들의 inertial을 origin rpy = "1.57075 1.57075 0" 으로 설정해주세요.  
origin은 위 그림에서 보신 것과 같이 joint 할 위치라고 생각하면 편할 것 같습니다.  

```xml
<?xml version="1.0"?>
<robot name="simplecar">
    <!-- Base Frame of Car -->
    <link name="base_link">
        <visual>
            <geometry>
                <box size="0.5 0.3 0.1"/>
            </geometry>
        </visual>
    </link>

    <!-- Left Back Wheel -->
    <link name="left_back_wheel">
        <visual>
            <geometry>
                <cylinder length="0.05" radius="0.1"/>
            </geometry>
            <origin rpy="1.57075 1.57075 0"/>
        </visual>
    </link>

    <joint name="base_to_left_back_wheel" type="continuous">
        <parent link="base_link"/>
        <child link="left_back_wheel"/>
        <axis xyz="0 1 0"/>
        <origin xyz="-0.2 0.175 0"/>
    </joint>
</robot>
```

#### 💻 연습코딩
오른쪽 바퀴와 마찬가지로 적용해봅시다.

##### Right Back Wheel
* Joint name: base_to_right_back_wheel
* Joint type: continuous
* link origin rpy="-1.57075 -1.57075 0"
* axis: 왼쪽 뒷바퀴와 동일
* origin xyz="-0.2 -0.175 0"

#### Material을 이용하여 색 입히기

이제는 각 geometry의 색을 입힐 것입니다. visual 하위 태그로, material id를 미리 설정하여 id로 간단하게 색을 넣을 수 있습니다.  
```texture name ... ``` 형태로 mtl 파일을 넣을 수도 있습니다.

``` xml
<?xml version="1.0"?>
<robot name="simplecar">
    <material name="blue">
        <color rgba="0.6 0.7 0.8 0.7"/>
    </material>

    <!-- Base Frame of Car -->
    <link name="base_link">
        <visual>
            <geometry>
                <box size="0.5 0.3 0.1"/>
            </geometry>
            <material name="blue"/>
        </visual>
    </link>
    ...
</robot>
```

#### 💻 연습코딩
모든 바퀴에 색깔을 입혀봅시다.

##### All Wheels
먼저 material name을 black으로 설정합니다. rgba 순서대로 0 0 0 1으로 설정하고 바퀴의 visual 태그 하위로 material을 설정합니다.

#### Physical attribute 적용하기

collision 부분은 위에서 언급되었듯, collosion box를 설정하게 하는 태그입니다.   
link의 하위 태그로 보통 geometry와 같게 설정됩니다.  
Inertial 부분은 건너 뛰겠습니다..!! (무게 설정하는 것만 확인..!)

```xml
<!-- Left Front Wheel -->

<link name="left_front_wheel">
    <visual>
        <geometry>
            <cylinder length="0.05" radius="0.1"/>
        </geometry>
        <origin rpy="1.57075 1.57075 0"/>
        <material name="black"/>
    </visual>

    <collision>
         <geometry>
            <cylinder length="0.05" radius="0.1"/>
        </geometry>
        <origin rpy="1.57075 1.57075 0"/>
    </collision>

    <inertial>
        <origin rpy="1.57075 1.57075 0"/>
        <mass value="0.3"/>
        <inertia ixx="0.4" ixy="0" ixz="0" iyy="0.4" iyz="0.0" izz="0.2"/>
    </inertial>
</link>
```

### 자동차 움직여보기

In [2]:
import pybullet as p
import pybullet_data
import pathlib as path

In [ ]:
client = p.connect(p.GUI)       #connection for visualization
                                #p.DIRECT: non-visual

p.setGravity(0,0,-10)
p.setAdditionalSearchPath(pybullet_data.getDataPath())

In [ ]:
angle =p.addUserDebugParameter("Steering",-0.5,0.5,0)
throttle = p.addUserDebugParameter("Throttle",0,20,0)

In [ ]:
# pybullet_data: urdf files
planeId = p.loadURDF("plane.urdf")
carPath = "D:\\GitHub\\Curling_technician\\pybullet_test\\model\\"
carId = p.loadURDF(carPath +"simplecar.urdf",basePosition=[0,0,0.2])
number_of_joints=p.getNumJoints(carId)

In [ ]:
wheel_indices = [1,3,4,5]
hinge_indicies = [0,2]

In [ ]:
while True:
    user_angle = p.readUserDebugParameter(angle)
    user_throttle = p.readUserDebugParameter(throttle)
    for joint_index in wheel_indices:
        p.setJointMotorControl2(carId, joint_index,
                                p.VELOCITY_CONTROL,
                                targetVelocity=user_throttle)

    for joint_index in hinge_indicies:
        p.setJointMotorControl2(carId, joint_index,
                                p.POSITION_CONTROL,
                                targetPosition=user_angle)

    p.stepSimulation()

#for _ in range(number_of_joints):
#    info = p.getJointInfo(carId, _)
#    position, orientation = p.getBasePositionAndOrientation(carId)
#    p.applyExternalForce(carId,0,[50,0,0],position,p.WORLD_FRAME)
#    p.stepSimulation()
#    print(info[0],":",info[1])

<center><em>For more information visit:</em></center>  

[![GitHub](http://img.shields.io/badge/-Tech%20blog-black?style=flat-square&logo=github&link=https://github.com/ameliacode)](https://github.com/ameliacode)